In [1]:
import json

In [17]:
with open('../data/WikiHow_filtered_data.json', 'r') as f:
    instructions = json.load(f)

with open('../data/Wikihow_time_appropriate.json', 'r') as f:
    times = json.load(f)

with open('../data/WikiHow_setting.json', 'r') as f:
    settings = json.load(f)

In [25]:
question = 'How To Make Chicken Cutlets'
idx = '0'
print(question)
print('\n')
print('\n'.join(instructions[question][idx]))
print('\n')
time_list = [str(i) for i in times[question][idx]]
print('\n'.join(time_list))
# idx = '1'
# print('\n'.join(instructions[question][idx]))

How To Make Chicken Cutlets


1. Slice the chicken breasts horizontally to 1/4 inch thickness.
2. Combine bread crumbs, Parmesan cheese, parsley, salt, and pepper in a bowl.
3. Beat the egg and milk together in a separate bowl.
4. Spread flour onto a plate.
5. Dredge each cutlet in flour, then the egg mixture, and then the bread crumb mixture.
6. Roast the breaded chicken cutlets.


8
3
2
1
6


In [2]:
with open('../data/Wikihow_time_reanalyse.json', 'r') as f:
    reanalyze = json.load(f)
    
with open('../data/WikiHow_filtered_data.json', 'r') as f:
    data = json.load(f)
    
with open('../data/Wikihow_time_None.json', 'r') as f:
    time_none = json.load(f)

In [8]:
cnt = 18

In [18]:
cnt += 1
tmp_cnt = 0
for key, value in reanalyze.items():
    tmp_cnt += 1
    if cnt > tmp_cnt:
        continue
    else:
        print(key)
        if len(value) == 0:
            continue
        print(value)
        print(data[key])
        print(time_none[key])
        break

How To Shrink Skinny Jeans
{'0': 'None', '1': 'None'}
{'0': ['1. Set the washing machine to high heat.', '2. Wash the skinny jeans alone in the washing machine.', '3. Dry the jeans on high heat.', '4. Repeat steps 1.1-1.3 if the jeans are still too big.'], '1': ['1. Boil a large pot of water.', '2. Turn off the heat and submerge the jeans in the boiling water.', '3. Let the jeans soak until the water cools.', '4. Dry the jeans on high heat.', '5. Repeat steps 2.1-2.4 if the jeans are still too big.']}
{'0': 'Step 1: 2 minutes\nStep 2: 45 minutes\nStep 3: 45 minutes\nStep 4: None', '1': 'Step 1: 15 minutes\nStep 2: 2 minutes\nStep 3: 60 minutes\nStep 4: 45 minutes\nStep 5: None'}
